In [1]:
# Датасет AI/ML Salaries https://www.kaggle.com/datasets/cedricaubin/ai-ml-salaries

import pandas as pd

# Для анализа исключим из датасета название работы, оставим зарплату только в долларах США,
# а также местоположение работника и компании
# Атрибуты experience_level, employment_type и company_size переведем в числовой формат

csv = pd.read_csv('salaries.csv', sep=',')
df = pd.DataFrame(csv)[['work_year', 'experience_level', 'employment_type', 
                        'salary_in_usd', 'remote_ratio', 'company_size'#, 'company_location'
                       ]]

df['experience_level'] = df['experience_level'].replace({'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3})
df['employment_type'] = df['employment_type'].replace({'PT': 0, 'FT': 1, 'CT': 2, 'FL': 3})
df['company_size'] = df['company_size'].replace({'S': 0, 'M': 1, 'L': 2})
#df = pd.concat([df, pd.get_dummies(df['company_location'], prefix='loc')],axis=1)
#df.drop(['company_location'],axis=1,inplace=True)
df.sample(5)

,work_year,experience_level,employment_type,salary_in_usd,remote_ratio,company_size
2052,2022,2,1,160000,100,1
2206,2022,1,1,54634,100,0
459,2023,0,1,60000,100,1
1276,2022,1,1,124000,100,1
239,2023,2,1,15827,100,0


In [2]:
# Поиск пропущенных значений
print('Пропущенные значения:')
for col in df.columns:
    num_missing = df[col].isnull().sum()
    if num_missing > 0:
        print(f'{col}: {num_missing}')
        
# Пропущенные значения отсутствуют

Пропущенные значения:


In [3]:
# Заменим значения зарплаты ниже 80000 на 0, а остальные на 1
df['salary_in_usd'] = (df['salary_in_usd'] >= 80000).astype(int)

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

target = 'salary_in_usd'
x = df.drop(target, axis=1)
y = df[target]

# Разделим датасет на тренировочные и тестировочные данные в соотношении 70%/30%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

tree = DecisionTreeClassifier(max_depth=5, random_state=42)
knn = KNeighborsClassifier(n_neighbors=10)

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Определим оптимальные параметры для дерева решений
tree_params = {"max_depth": range(1, 11), "max_features": range(4, 19)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(x_train, y_train)
tree_grid.best_params_, tree_grid.best_score_

Fitting 5 folds for each of 150 candidates, totalling 750 fits


({'max_depth': 5, 'max_features': 5}, 0.8459981902499717)

In [5]:
from sklearn.preprocessing import StandardScaler

# Определим оптимальные параметры для k-NN
knn_pipe = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_jobs=-1))])
knn_params = {"knn__n_neighbors": range(1, 10)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(x_train, y_train)
knn_grid.best_params_, knn_grid.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


({'knn__n_neighbors': 9}, 0.8326931342608302)

In [6]:
from sklearn.metrics import accuracy_score

# Определим долю правильных ответов алгоритмов
print('Accuracy')
print(f'k-NN: {accuracy_score(y_test, knn_grid.predict(x_test))}')
print(f'Дерево решений: {accuracy_score(y_test, tree_grid.predict(x_test))}')

Accuracy
k-NN: 0.8215077605321508
Дерево решений: 0.8470066518847007


In [7]:
from sklearn.metrics import precision_score

# Определим точность алгоритмов
print('Presicion')
print(f'k-NN: {precision_score(y_test, knn_grid.predict(x_test))}')
print(f'Дерево решений: {precision_score(y_test, tree_grid.predict(x_test))}')

Presicion
k-NN: 0.8730366492146597
Дерево решений: 0.8665018541409147


In [8]:
from sklearn.metrics import recall_score

# Определим полноту алгоритмов
print('Recall')
print(f'k-NN: {recall_score(y_test, knn_grid.predict(x_test))}')
print(f'Дерево решений: {recall_score(y_test, tree_grid.predict(x_test))}')

Recall
k-NN: 0.9124487004103967
Дерево решений: 0.9589603283173734


In [9]:
from sklearn.metrics import f1_score

# Определим F-меру алгоритмов
print('F-measure')
print(f'k-NN: {f1_score(y_test, knn_grid.predict(x_test))}')
print(f'Дерево решений: {f1_score(y_test, tree_grid.predict(x_test))}')

F-measure
k-NN: 0.8923076923076924
Дерево решений: 0.9103896103896104


In [10]:
from sklearn.metrics import roc_auc_score

# Определим площадь под ROC-кривой
print('ROC')
print(f'k-NN: {roc_auc_score(y_test, knn_grid.predict(x_test))}')
print(f'Дерево решений: {roc_auc_score(y_test, tree_grid.predict(x_test))}')

ROC
k-NN: 0.6725986192110462
Дерево решений: 0.6636906904744762


In [11]:
# В каких метриках каждый алгоритм показал себя лучше:
# k-NN: presicion, ROC
# Дерево решений: accuracy, recall, F-measure

# Более высокие значения ROC AUC связаны с тем, что эта метрика является гармоническим средним между presicion и recall

# По совокупности всех параметров можно сделать вывод о том, что дерево решений является наиболее подходящем алгоритмом 
# с подобными входными данными

# Основным преимуществом дерева решений над k-NN является более высокая доля правильных ответов: 84.7% против 82.2%